<a href="https://colab.research.google.com/github/BrendaOliveiira/Automacao/blob/main/Automatizacao_SG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

In [28]:
!pip install openpyxl

In [29]:
df_base = pd.read_excel('/content/Base Global Consolidada - 03_2024 (Automática).xlsx', sheet_name="Base", header=2)

In [ ]:
df_base = df_base.drop(columns=['ANOMES','B8_EMPRESA','DESC_EMPRESA','B8A_CNPJ_EMPRESA','C6A_CNPJ_LOCAL','D6_CENTRO_LUCRO','B9_UNIDADE','UNIDADE_PES','SG_UNID_ORGN','CCUSTO_DESC','C4_GESTOR_MEDIATO','A6_NUMERO_DOCUMENTO','RG','IDADE','GERACAO','FAIXA_ETÁRIA','B5_PRIMEIRA_ADMISSAO','B6_DATA_DESLIGAMENTO','B7_MOTIVO_DESLIGAMENTO','TEMPO','FAIXA TEMPO NA EMPRESA','CODIGO_CARGO','B3_GRADE_NOVO','CARGO_LIDERANCA','DATA_ASSUMIU_CARGO','TEMPO_CARGO','FAIXA_TEMPO_CARGO','C91_ULTIMA_MOVIMENTACAO_DT','C92_DESC_MOTIVO','C10_PERCENTUAL','D1_SALARIO_MENSAL','D1_SALARIO_HORA','%_DA_FAIXA','HORASMES','D4_PERC_REGIAO_TRANSF','AG_LOCAL','E5_CONTRATO_SUSPENSO','E6_EXPATRIADO','A4_FORMACAO','DESC_RACA_COR','DESC_PCD','LOCAL_HIST','DESC_LOCAL','UF_LOCAL','SINDICATO','SINDICATO_DESC','CA_MAPR','SALBAS_PORTAL','RES_DDD','RES_FONE','DDD_CELULAR','CELULAR','EMAIL_ALTER','TEMPO_DE_CASA_ULTIMA_ADMISSAO','FAIXA_DE_TEMPO_ULTIMA_ADMISSAO','POSICAO_GRADE_FINAL ','ID DUPLICADO','RETIRAR DA CONTAGEM QUANDO ID DUPLICADO( CONSIDERA ID CONSAG)','CATEGORIA_AREA','NÍVEL DO CARGO','BP FAZ GESTAO','E3_ESCOLARIDADE','PUBLICO OVERHEAD']).copy()
df_base

In [ ]:
df_base = df_base.rename(columns={'C2_UN_DIRETORIA':'LOCATION','C1_AREA':'DEPARTMENT','CCUSTO':'CUSTOM01', 'C3_GESTOR_IMEDIATO':'MANAGER','C6_ESCRITORIO_OBRA':'ADDR1','ID_UNICO':'USERID','B1_MATRICULA':'EMPID','A1_NOME':'NOME','A1_EMAIL':'EMAIL','A2_GENERO':'GENDER','A3_DATA_NASCIMENTO':'CUSTOM13','E1_STATUS':'CUSTOM08','B4_DATA_ADMISSAO':'HIREDATE','B2_CARGO':'TITLE','E2_CATEGORIA':'CUSTOM02','DESC_CONS_GENTE':'HR','GRUPO':'DIVISION','A7_NACIONALIDADE':'CUSTOM04'}).copy()
df_base

In [ ]:
def limpar_texto(texto):
    texto_limpo = texto.replace('_x000D_\n', '').strip()
    texto_limpo = texto_limpo.strip('"')
    return texto_limpo
df_base['HR'] = df_base['HR'].apply(lambda x: limpar_texto(x))
df_base

In [ ]:
df_base['STATUS'] = 'Active'
df_base['TIMEZONE'] = 'America/Sao_Paulo'
df_base['COUNTRY'] = 'BR'
df_base['DEFAULT_LOCALE'] = 'pt_BR'
df_base['LOGIN_METHOD'] = 'SSO'
df_base['JOBCODE'] = df_base['TITLE']
df_base['MATRIX_MANAGER'] = df_base['MANAGER']
df_base

In [ ]:
df_base[['FIRSTNAME','LASTNAME']] = df_base['NOME'].str.split(" ", n=1, expand=True)
df_base

In [ ]:
nova_ordem_colunas = ['STATUS','USERID','NOME','FIRSTNAME','LASTNAME','GENDER','EMAIL','MANAGER','HR','DIVISION','DEPARTMENT','LOCATION','JOBCODE','TIMEZONE','HIREDATE','EMPID','TITLE','ADDR1','COUNTRY','CUSTOM01','CUSTOM02','CUSTOM04','CUSTOM08','CUSTOM13','MATRIX_MANAGER','DEFAULT_LOCALE','LOGIN_METHOD']
df_base = df_base.reindex(columns=nova_ordem_colunas)
df_base

In [ ]:
userids = ['NOME','USERID']
df_users = df_base[userids].copy()
df_users

In [ ]:
df_base = df_base.merge(df_users, left_on='HR',right_on='NOME',how='left')
df_base

In [ ]:
df_base['HR'] = df_base['USERID_y']
df_base

In [ ]:
df_base = df_base.drop(columns=['NOME_x','NOME_y','USERID_y']).copy()
df_base

In [ ]:
df_base = df_base.rename(columns={'USERID_x':'USERID'}).copy()
df_base

In [ ]:
df_base['MANAGER'] = df_base['MANAGER'].str.split().str[0]
df_base['MATRIX_MANAGER'] = df_base['MANAGER']
df_base

In [ ]:
df_base['USERNAME'] = df_base['EMAIL'].str.split('@').str[0]
df_base

In [ ]:
nova_ordem_colunas = ['STATUS','USERID','USERNAME','FIRSTNAME','LASTNAME','GENDER','EMAIL','MANAGER','HR','DIVISION','DEPARTMENT','LOCATION','JOBCODE','TIMEZONE','HIREDATE','EMPID','TITLE','ADDR1','COUNTRY','CUSTOM01','CUSTOM02','CUSTOM04','CUSTOM08','CUSTOM13','MATRIX_MANAGER','DEFAULT_LOCALE','LOGIN_METHOD']
df_base = df_base.reindex(columns=nova_ordem_colunas)
df_base

In [ ]:
df_base = df_base[(df_base['DIVISION'] != 'LOCAL') & (df_base['DIVISION'] != 'AFINIDADE BRADESCO')]
df_base

In [ ]:
valores_excluir = ['AFAST_ACIDENTE_TRABALHO', 'AFAST_APOS_PREVIDENCIA', 'AFAST_APOS_ACIDENTE', 'AFAST_AUXILIO_DOENCA', 'DEMITIDO', 'DEMITIDO_2VINC', 'AFAST_LIC_MATERNIDADE', 'MES_AUXILIO_DOENCA', 'AFAST_LIC_REMUNERADA','AFAST_CONTRATO_SUSPENSO', 'MES_LICENCA_REMUNERADA']
df_base = df_base[~df_base['CUSTOM08'].isin(valores_excluir)]
df_base


In [ ]:
df_base['CUSTOM08'] = 'Ativo'
df_base

In [ ]:
valores_excluir = ['AUXILIAR DE SERVICOS GERAIS', 'COPEIRA', 'ESTAGIARIO', 'JOVEM APRENDIZ', 'MENSAGEIRO', 'MOTORISTA']
df_base = df_base[~df_base['JOBCODE'].isin(valores_excluir)]
df_base

In [ ]:
df_base['Check_Manager'] = df_base['MANAGER'].isin(df_base['USERID'])
df_base['Check_Manager'] = df_base['Check_Manager'].map({True:'Ativo',False:'Desligado'}).copy()
df_base

In [ ]:
df_base_manager_desligado = df_base[df_base['Check_Manager'] == 'Desligado']
df_base_manager_desligado

In [52]:
df_base_manager_desligado.to_excel('Gestor_Desligado.xlsx', index=False)

In [51]:
df_base.to_excel('Base Sistema de Gente.xlsx', index=False)